# 多个输入通道
彩色图像可能有RGB三个通道
转换为灰度会丢失信息

# 多个输入通道
每个通道都有一个卷积核，结果是所有通道卷积结果的和
 ·输入$\mathbf{X}:c_i\times n_h\times n_w$
 ·核$\mathbf{W}:c_i\times k_h\times k_w$
 ·输出$\mathbf{Y}:m_{h}\times m_{w}$
 $$\mathbf{Y}=\sum_{i=0}^{c_i}\mathbf{X}_{i,:,:}\star\mathbf{W}_{i,:,:}$$

# 多个输出通道
无论有多少输入通道，到目前为止我们只用到单输出通道
我们可以有多个三维卷积核，每个核生成一个输出通道

· 输入$\mathbf{X}:c_i\times n_h\times n_w$
·核$\mathbf{W}:c_{o}\times c_{i}\times k_{h}\times k_{w}$
·输出$\mathbf{Y}:c_o\times m_h$文$m_{w}$
$$\mathbf{Y}_{i,:,:}=\mathbf{X}\star\mathbf{W}_{i,:,:,:}\quad\text{for}i=1,...,c_o$$

# 多个输入和输出通道
·每个输出通道可以识别特定模式
输入通道核识别并组合输入中的模式

# 1x1卷积层
$k_{h}=k_{w}=1$是一个受欢迎的选择。 它不识别空间模式，只是融合通道。
相当于输入形状为$n_hn_w\times c_i$,权重为$c_o\times c_i$的

# 二维卷积层
·输入$\mathbf{X}:c_{i}\times n_{h}\times n_{w}$
·核$\mathbf{W}:c_{o}\times c_{i}\times k_{h}\times k_{w}$
·偏差 B : $c_{o}\times c_{i}$
·输出$\mathbf{Y}:c_{o}\times m_{h}\times m_{w}$
·计算复杂度 (浮点计算数 FLOP) $O(c_ic_ok_hk_wm_hm_w)$
$$\mathbf{Y=X}\star\mathbf{W}+\mathbf{B}$$

$$\begin{array}{c}c_i=c_o=100\\k_h=h_w=5\\m_h=m_w=64\end{array}$$
$$\Downarrow$$
$$1GFLOP$$
10层，1M 样本，10 PFlops(CPU: 0.15 TF=18h,GPU: 2 TF=14min)

# 总结
输出通道数是卷积层的超参数
每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果
每个输出通道有独立的三维卷积核

# 多输入多输出通道
实现一下多输入通道互相关运算

In [2]:
import torch
from d2l import torch as d2l


def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证互相关运算的输出

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多通道的输出的互相关函数

In [4]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)


K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1×1卷积

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6